In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
answer_df = pd.read_csv('./두통 답변.csv', index_col=0)
question_df = pd.read_csv('./두통 질문.csv', index_col=0)

In [3]:
answer_df

,disease_category,disease_name,intention,answer_intro_conclusion,answer
0,응급질환,두통,예방,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...
1,응급질환,두통,예방,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 마지막으로, 의사나...","긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활..."
2,응급질환,두통,예방,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...
3,응급질환,두통,예방,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 이러한 전략들은 ...,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...
4,응급질환,두통,예방,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...
...,...,...,...,...,...
5513,응급질환,두통,치료,"두통은 우리 일상생활에서 흔히 겪는 증상입니다. 만약 두통이 반복적으로 발생한다면,...",두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...
5514,응급질환,두통,치료,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...
5515,응급질환,두통,치료,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...
5516,응급질환,두통,치료,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...


### 키워드 및 문장기반 유사도 측정 및 질문-답변 매칭
(1) KeyBERT 모델을 사용해 키워드 유사도가 높은 질문들만 1차 추출
- 명사, 형용사 형태소로만 재 문장 구성 
- KeyBERT 모델을 활용해 문장의 키워드 추출(BERT 기반이므로 문맥정보 이해)
- TF-IDF 벡터로 변환 후 코사인 유사도 계산. 답변당 상위 n개의 질문만 추출
- 바로 모든 질문과 답변을 비교하기보다 1차 필터링을 통해 문장수를 줄여 대용량시 연산 속도를 높이기 위함.

(2) 필터링된 문장들만 임베딩 후 유사도 비교
- KoBERT: 한국어로 된 문장, 단어, 문맥을 이해하고 처리하는 범용 모델. ex) 감정 분석, 문서 분류, 질의 응답 등
- KR-SBERT: 문장 유사도에 조금 더 최적화

(3) 가장 연관성이 높은 질문 데이터프레임화

### (1)-1 형태소 분석 후 문장 재생성

In [4]:
from konlpy.tag import Okt

okt = Okt()

# 형태소 분석 및 품사 정보
test_keyword = []
for sentence in question_df['question']:
    test_keyword.append(okt.pos(sentence))

- 키워드 추출, 문장 재생성 함수

In [5]:
def text_extract(df, column):
    # 의도
    pos_keywords = []
    for q in df[f'{column}']:
        pos_keywords.append(okt.pos(q))

    # Noun, Adjective 키워드만 추출
    keyword_lists = []
    for list in pos_keywords:
        
        sentence_keywords = []
        for pair in list:
            keyword, pos = pair
            if pos in ['Noun', 'Adjective']:
                sentence_keywords.append((keyword, pos))

        keyword_lists.append(sentence_keywords)

    return keyword_lists

In [6]:
question_keyword_lists = text_extract(question_df, 'question')

In [7]:
# 다시 문장으로 만드는 함수 만들기
def remake_sentence(keyword_lists):
    remake_question = []
    for list in keyword_lists:
        sentence = ''
        for pair in list:
            sentence = sentence + pair[0] + ' '
        remake_question.append(sentence)

    return remake_question

In [8]:
question_remake_sentence = remake_sentence(question_keyword_lists)
question_remake_sentence[:5]

['두통 예방 위해 어떤 자세 활동 도움 요 ',
 '스트레스 사람 두통 예방 위해 어떤 생활 습관 요 ',
 '대상포진 예방접종 후 어지럼증 두통 예방접종 관련 있다면 어떻게 ',
 '갈색 포증 관련 일상생활 신경 점 무엇 ',
 '두통 예방 위 수면 자세 어떤 것 있을까요 ']

- 의도별 키워드 추출 및 문장 재생성 함수로 문장 재생성하기

In [9]:
question_df['intention'].unique()

array(['예방', '원인', '정의', '증상', '진단', '치료'], dtype=object)

In [10]:
# 키워드 추출 및 문장 재생성 함수
def remake_sentence_by_intention(df, intention, column):
    filtered_df = df[df['intention'] == f'{intention}']
    # 형태소 키워드 추출
    keyword_lists = text_extract(filtered_df, column)
    # 문장 재생성
    result = remake_sentence(keyword_lists)

    return result

In [11]:
len(remake_sentence_by_intention(question_df, '예방', 'question'))

308

In [12]:
intention_en = {'예방': 'prevention',
                '원인': 'cause',
                '정의': 'definition',
                '증상': 'symptom',
                '진단': 'diagnosis',
                '치료': 'treatment'}

for i in question_df['intention'].unique():
    print(f'{intention_en[i]}_sentence')

prevention_sentence
cause_sentence
definition_sentence
symptom_sentence
diagnosis_sentence
treatment_sentence


In [13]:
# 의도별 질문 문장 재생성
question_sentences_dic = {} # 딕셔너리로 변수명 저장

for i in question_df['intention'].unique():
    var_name = f'{intention_en[i]}_sentence' # 변수명
    question_sentences_dic[var_name] = remake_sentence_by_intention(question_df, i, 'question')

In [14]:
question_sentences_dic['prevention_sentence'][:3]

['두통 예방 위해 어떤 자세 활동 도움 요 ',
 '스트레스 사람 두통 예방 위해 어떤 생활 습관 요 ',
 '대상포진 예방접종 후 어지럼증 두통 예방접종 관련 있다면 어떻게 ']

In [15]:
# 의도별 답변 문장 재생성
answer_sentences_dic = {} # 딕셔너리로 변수명 저장

for i in answer_df['intention'].unique():
    var_name = f'{intention_en[i]}_sentence' # 변수명
    answer_sentences_dic[var_name] = remake_sentence_by_intention(answer_df, i, 'answer_intro_conclusion')

In [16]:
answer_sentences_dic['prevention_sentence'][:3]

['두통 예방 위 일상 자세 유지 일상 생활 자세한 관리 두통 예방 도움 수 있습니다 올바른 자세 수면 스트레칭 통해 일상 생활 올바른 자세 유지 것 두통 예방 도움 ',
 '긴장 두통 예방 위 몇 가지 조치 취할 수 있습니다 마지막 의사 영양사 상담 통해 적절한 예방 조치 취하 것 중요합니다 의사 영양사 조언 정기 건강 관리 스트레스 관리 긴장 두통 예방 데 도움 것 입니다 ',
 '두통 예방 위 생활 습관 다양한 방법 개선 수 있습니다 일상 습관 중 하나 충분한 수면 취하 것 입니다 수면 시간 충분히 확보 잠자리 전 긴장 운동 활동 것 좋습니다 또한 스트레스 관리 것 중요한 요소 입니다 스트레스 긴장 두통 유발 주요한 요인 므 스트레스 효과 관리 방법 것 중요합니다 두통 예방 위 일상 습관 개선 스트레스 관리 충분한 휴식 필요합니다 스트레스 높을 경우 의사 상담 적절한 치료 방법 것 좋습니다 ']

### (1)-2 keybert 모델로 키워드 추출
- 재생산한 문장에서의 키워드 중요도 반영

In [17]:
from keybert import KeyBERT

# 모델 불러오기
kw_model = KeyBERT()

In [18]:
list(question_sentences_dic.keys())

['prevention_sentence',
 'cause_sentence',
 'definition_sentence',
 'symptom_sentence',
 'diagnosis_sentence',
 'treatment_sentence']

In [19]:
question_sentences_dic.keys()

dict_keys(['prevention_sentence', 'cause_sentence', 'definition_sentence', 'symptom_sentence', 'diagnosis_sentence', 'treatment_sentence'])

In [20]:
# 질문 keybert 키워드 추출
question_keybert_dic = {}

# 변수명 딕셔너리에 키값으로 담음.
for key in list(question_sentences_dic.keys()):
    var = key.split('_')[0]
    var_name = f'{var}_question_keywords'

    question_keybert_dic[var_name] = []
    # question_keywords = []
    for q in question_sentences_dic[key]:
        kw = kw_model.extract_keywords(q, keyphrase_ngram_range=(1, 1), top_n=10)
        question_keybert_dic[var_name].append(kw)

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [21]:
# 제대로 들어갔나 확인
question_keybert_dic.keys(), question_keybert_dic['treatment_question_keywords'][:2]

(dict_keys(['prevention_question_keywords', 'cause_question_keywords', 'definition_question_keywords', 'symptom_question_keywords', 'diagnosis_question_keywords', 'treatment_question_keywords']),
 [[('진통제', 0.6325),
   ('충분한', 0.5314),
   ('두통', 0.4342),
   ('치료', 0.3565),
   ('효과', 0.3234),
   ('있을까요', 0.201),
   ('있을', 0.201)],
  [('성적', 0.5468),
   ('달성', 0.4924),
   ('장기', 0.444),
   ('두통', 0.4321),
   ('목표', 0.4252),
   ('도움', 0.2602),
   ('전략', 0.1802),
   ('예방', 0.1802),
   ('어떤', 0.1802)]])

In [22]:
# 답변 keybert 키워드 추출
answer_keybert_dic = {}

# 변수명 딕셔너리에 키값으로 담음.
for key in list(answer_sentences_dic.keys()):
    var = key.split('_')[0]
    var_name = f'{var}_answer_keywords'

    answer_keybert_dic[var_name] = []
    for q in answer_sentences_dic[key]:
        kw = kw_model.extract_keywords(q, keyphrase_ngram_range=(1, 1), top_n=10)
        answer_keybert_dic[var_name].append(kw)

In [23]:
# 제대로 들어갔나 확인
answer_keybert_dic.keys(), answer_keybert_dic['treatment_answer_keywords'][:2]

(dict_keys(['prevention_answer_keywords', 'cause_answer_keywords', 'definition_answer_keywords', 'symptom_answer_keywords', 'diagnosis_answer_keywords', 'treatment_answer_keywords']),
 [[('중요합니다', 0.5574),
   ('정신', 0.4489),
   ('적절한', 0.4295),
   ('사용', 0.4231),
   ('요법', 0.4121),
   ('치료', 0.4052),
   ('결정', 0.4021),
   ('생활', 0.3958),
   ('이완', 0.3898),
   ('전문가', 0.385)],
  [('정확한', 0.2811),
   ('성적', 0.2523),
   ('적절한', 0.2416),
   ('건강한', 0.2302),
   ('통증', 0.2273),
   ('중요합니다', 0.2206),
   ('라이프스타일', 0.2107),
   ('증상', 0.2028),
   ('의해', 0.1943),
   ('정기', 0.1932)]])

#### (1)-3 TF-IDF 벡터로 변환 후 코사인 유사도 계산
- TF-IDF 사용 이유: 유사한 질문들이 있을경우 키워드간 중요도를 판별하기 위함.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 키워드를 TF-IDF 벡터로 변환
# 질문을 학습시킨 벡터화 모델에 답변을 벡터화해야 차원을 맞출 수 있음.
# 질문만 학습시켜 생긴 어휘사전 기반으로 답변은 변환만 하면 어휘사전에 없는 단어들은 벡터화가 안됨.
# 즉, 답변에만 등장하는 단어들의 유사도를 비교할 필요가 없는 것.
vectorizer = TfidfVectorizer()

def tfidf_vectorizer(q_keybert_dic, a_keybert_dic, q_column, a_column, vectorizer):
    
    q_tfidf_dic = {}
    a_tfidf_dic = {}
    for q_key, a_key in zip(q_keybert_dic.keys(), a_keybert_dic.keys()):
        var = q_key.split('_')[0]
        q_var_name = f'{var}_{q_column}_vectors'
        a_var_name = f'{var}_{a_column}_vectors'


        q_tfidf_dic[q_var_name] = vectorizer.fit_transform([' '.join([kw[0] for kw in kblist]) for kblist in q_keybert_dic[q_key]])
        a_tfidf_dic[a_var_name] = vectorizer.transform([' '.join([kw[0] for kw in kblist]) for kblist in a_keybert_dic[a_key]])

    return q_tfidf_dic, a_tfidf_dic

In [25]:
# tfi-df 벡터화 진행
question_vectors_dic, answer_vectors_dic = tfidf_vectorizer(question_keybert_dic, answer_keybert_dic, 'question', 'answer', vectorizer)

In [26]:
question_vectors_dic

{'prevention_question_vectors': <308x186 sparse matrix of type '<class 'numpy.float64'>'
 	with 2234 stored elements in Compressed Sparse Row format>,
 'cause_question_vectors': <567x260 sparse matrix of type '<class 'numpy.float64'>'
 	with 3556 stored elements in Compressed Sparse Row format>,
 'definition_question_vectors': <606x278 sparse matrix of type '<class 'numpy.float64'>'
 	with 3859 stored elements in Compressed Sparse Row format>,
 'symptom_question_vectors': <772x361 sparse matrix of type '<class 'numpy.float64'>'
 	with 5531 stored elements in Compressed Sparse Row format>,
 'diagnosis_question_vectors': <707x335 sparse matrix of type '<class 'numpy.float64'>'
 	with 5096 stored elements in Compressed Sparse Row format>,
 'treatment_question_vectors': <468x240 sparse matrix of type '<class 'numpy.float64'>'
 	with 3283 stored elements in Compressed Sparse Row format>}

In [27]:
print(answer_vectors_dic['symptom_answer_vectors'].shape) # 316개의 문장이 361개의 키워드로 표현됨.

(316, 361)


In [28]:
# 코사인 유사도 함수화
# 증상 질문과 증상 답변, 진단 질문과 진단 답변들 간의 유사도
# 코사인 유사도 결과 값을 보면 첫번째 데이터는 첫번째 답변과 모든 질문간의 유사도가 하나의 행렬 벡터에 있는 것이다.
# 즉 각 벡터마다 답변 문장의 수만큼 유사도 점수가 있음.
# 모든 답변 벡터에 대한 질문들의 유사도를 비교하고 싶으므로 답변 벡터를 먼저 배치.
def from_dic_cos_similarity(q_vector_dic, a_vector_dic):

    similarity_dic = {}
    
    for q_key, a_key in zip(q_vector_dic.keys(), a_vector_dic.keys()):
        var = q_key.split('_')[0]
        var_name = f'{var}_keyword_similarities'

        # 질문에 대한 
        similarity_dic[var_name] = cosine_similarity(a_vector_dic[a_key], q_vector_dic[q_key])
    
    return similarity_dic

In [29]:
keyword_similarities_dic = from_dic_cos_similarity(question_vectors_dic, answer_vectors_dic)

In [30]:
keyword_similarities_dic['prevention_keyword_similarities'][:20]

array([[0.13805125, 0.13321107, 0.00853558, ..., 0.01095506, 0.01342833,
        0.01021405],
       [0.        , 0.        , 0.        , ..., 0.23342105, 0.        ,
        0.        ],
       [0.        , 0.27651168, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.15550197, 0.        , ..., 0.42014554, 0.        ,
        0.        ],
       [0.        , 0.22172608, 0.        , ..., 0.34910328, 0.        ,
        0.        ],
       [0.01590486, 0.3341344 , 0.01293952, ..., 0.01660733, 0.02035669,
        0.015484  ]])

In [31]:
# 모든 답변에 대한 질문을 매칭해야 하므로 답변의 개수만큼 유사도 리스트가 생성되야함.

print("데이터 프레임 '예방' 답변 개수: ", len(answer_df[answer_df['intention'] == '예방']))
print("'예방' 유사도 리스트 개수: ", len(keyword_similarities_dic['prevention_keyword_similarities']))

데이터 프레임 '예방' 답변 개수:  1108
'예방' 유사도 리스트 개수:  1108


In [32]:
# 각 증상 질문당 매칭된 답변의 유사도 개수
print('첫번째 답변에 매칭된 질문 유사도 개수: ', len(keyword_similarities_dic['prevention_keyword_similarities'][0]))
print('두번째 답변에 매칭된 질문 유사도 개수: ', len(keyword_similarities_dic['prevention_keyword_similarities'][1]))

첫번째 답변에 매칭된 질문 유사도 개수:  308
두번째 답변에 매칭된 질문 유사도 개수:  308


In [33]:
len(keyword_similarities_dic['prevention_keyword_similarities'])

1108

In [34]:
k = list(keyword_similarities_dic.keys())
for i in k:
    print(len(keyword_similarities_dic[i]))

1108
1328
1375
316
1231
160


In [35]:
k = list(keyword_similarities_dic.keys())
for i in k:
    if i == k[0]:
        print('ddd')
    else:
        print(i)

ddd
cause_keyword_similarities
definition_keyword_similarities
symptom_keyword_similarities
diagnosis_keyword_similarities
treatment_keyword_similarities


In [118]:
def top_n_similarity_idx(similarity_dic, top_n=5):

    # key 리스트를 먼저 만들어야 마지막 for문에 사용할 수 있음.
    key_lists = list(similarity_dic.keys())
    answer_total_num = 0
    question_total_num = 0
    final_candidate_pairs =[]
    
    for k in key_lists:
        
        candidate_pairs = []
        
        for i, row in enumerate(similarity_dic[k]):
            idx_row=[]
            for j, score in enumerate(row):
                # 유사도와 인덱스를 튜플로 만들어서 저장
                idx_row.append((j, score))
                # reverse=True: 내림차순
                # key: x[1]에 있는 값(=score)을 기준으로 정렬
            
            top_sim = sorted(idx_row, reverse=True, key=lambda x: x[1])[:top_n]
            for j, score in top_sim:
                if k == key_lists[0]:
                    candidate_pairs.append((i, j))
                else:
                    candidate_pairs.append((i + answer_total_num, j + question_total_num))
                
        answer_total_num += len(similarity_dic[k])
        question_total_num += len(similarity_dic[k][0])
        final_candidate_pairs += candidate_pairs
                    

    return final_candidate_pairs

In [119]:
final_candidate_pairs = top_n_similarity_idx(keyword_similarities_dic, top_n=20)

In [38]:
print('원천데이터 답변 개수: ', len(answer_df))
print('최종 유사도 인덱스 쌍의 개수/n개: ', len(final_candidate_pairs) / 20)

원천데이터 답변 개수:  5518
최종 유사도 인덱스 쌍의 개수/n개:  5518.0


In [39]:
len(final_candidate_pairs)

110360

In [120]:
# 답변당 20개씩 매칭이 되어있음.
# (답변 위치(몇 번째 답변), 질문 위치)
final_candidate_pairs[22140:22170]

[(1107, 51),
 (1107, 88),
 (1107, 87),
 (1107, 16),
 (1107, 114),
 (1107, 290),
 (1107, 266),
 (1107, 197),
 (1107, 166),
 (1107, 220),
 (1107, 115),
 (1107, 77),
 (1107, 30),
 (1107, 222),
 (1107, 195),
 (1107, 1),
 (1107, 61),
 (1107, 74),
 (1107, 280),
 (1107, 300),
 (1108, 393),
 (1108, 519),
 (1108, 569),
 (1108, 448),
 (1108, 467),
 (1108, 512),
 (1108, 328),
 (1108, 381),
 (1108, 460),
 (1108, 370)]

### (2) 필터링된 문장들만 문장 임베딩
- KoBert 모델 사용

In [41]:
import torch
from kobert_transformers import get_kobert_model, get_tokenizer
model = get_kobert_model()
tokenizer = get_tokenizer()

# tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

In [96]:
# 임베딩 구하는 함수 만들기
def embedding(sentence):
    # BERT 모델 최대 입력길이 제한이 512 토큰 이라고 함.
    tokens = tokenizer.encode(sentence, return_tensors='pt', max_length=512, truncation=True, padding=True)

    # 추론과정에서 기울기 구하지 않도록 설정
    # 학습이 아니라 추론만 하므로 메모리 사용량을 줄이고 속도를 높이기 위해 no_grad 사용
    with torch.no_grad():
        output = model(tokens)


    # output[0]: 각 입력 토큰에 대한 벡터
    # shape: [batch_size, seq_length, hidden_size]
    # output[1]: 문장 전체적인 의미
    # [:, 0, :]: 모든 batch 차원(한번에 처리하는 데이터 묶음), 첫번째 토큰, 마지막 차원 모두
    return output[0][:, 0, :].squeeze().numpy() # 유사도 비교를 위해 numpy 배열로 변환

In [97]:
# 인트로+결론 답변, 전체 답변, 질문에 대한 임베딩 계산
question_embeddings_kobert = [embedding(q) for q in question_df['question']] 

In [44]:
ic_answer_embeddings_kobert = [embedding(a) for a in answer_df['answer_intro_conclusion']]

In [45]:
answer_embeddings_kobert = [embedding(a) for a in answer_df['answer']]

In [46]:
len(answer_embeddings_kobert), len(ic_answer_embeddings_kobert), len(answer_embeddings_kobert)

(5518, 5518, 5518)

In [47]:
# 후보 쌍에 대한 코싸인 유사도 계산 함수

def make_cosine_similarity(candidate_pairs, answer_embedding, question_embedding):
    
    similarities = []
    for a_idx, q_idx in candidate_pairs:
        # print(a_idx, q_idx)
        c_simi = cosine_similarity([answer_embedding[a_idx]], [question_embedding[q_idx]])
        similarities.append((a_idx, q_idx, c_simi))

    return similarities

In [48]:
# 전체 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
kobert_final_similarities = make_cosine_similarity(final_candidate_pairs, answer_embeddings_kobert, question_embeddings_kobert)

In [49]:
len(kobert_final_similarities), kobert_final_similarities[1900]

(110360, (95, 162, array([[0.60487926]], dtype=float32)))

In [50]:
# intro+conclusion 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
ic_kobert_final_similarities = make_cosine_similarity(final_candidate_pairs, answer_embeddings_kobert, question_embeddings_kobert)

- 가중치를 부여한 유사도 점수 계산
- intro + conclusion 답변에 대한 유사도에 가중치 0.4, 전체 답변에 대한 유사도에 가중치 0.6 부여

In [52]:
ic_kobert_final_similarities[-1]

(5517, 3206, array([[0.6336633]], dtype=float32))

In [53]:
combine_similarities = []
for ic_s, s in zip(ic_kobert_final_similarities, kobert_final_similarities):
    a, b, ic_c = ic_s
    _, _, c = s

    combine_score = 0.4*ic_c + 0.6*c

    combine_similarities.append((a, b, combine_score))

kobert_final_similarities = combine_similarities

In [58]:
kobert_final_similarities[-1]

(5517, 3206, array([[0.6336633]], dtype=float32))

### (3) 가장 연관성이 높은 질문 데이터 프레임화

- 10만개 이상의 데이터를 계속 for문 속에 for문을 넣으면서 코드를 돌리니 시간이 너무 오래걸림.
- defaultdict를 사용해 한 번만 순회하면서 최대유사도를 업데이트하는 방향으로 코드 변경
- 160분 넘어도 안끝나던 코드가 -> 0.2초만에 끝남
- defaultdict
    - dictionary와 동일하게 key: value 형식.
    - 존재하지 않는 key에 대해 기본값을 설정할 수 있음.

In [82]:
# # 인덱스와 유사도 리스트화
# idx_group = set(i[0] for i in kobert_final_similarities) # set(): 유니크한 값을 가지는 집합

# for g in idx_group:
#     simi_list = [i[2] for i in kobert_final_similarities if i[0] == g]

#     # 유사도가 가장 높은 값에 해당하는 데이터 하나만 추출
#     result = [row for row in kobert_final_similarities if (row[2] == max(simi_list)) and (row[0]==g)] 

In [76]:
from collections import defaultdict
a = defaultdict(int)

In [77]:
a[0], a['키값']

(0, 0)

In [98]:
from collections import defaultdict

# 키에 대한 기본값 설정
# 어떤 키를 넣든 없으면 해당 기본값을 줌
max_group = defaultdict(lambda: (-1, None)) # max_group에 처음 접근할 때 키가 없으면 (-1(유사도), None(원래값))이 기본값

for row in kobert_final_similarities:
    a_idx, q_idx, simi = row
    if simi > max_group[a_idx][0]: # simi > 현재 할당되어있는 유사도 이면
        max_group[a_idx] = (simi, row) # 동일한 a_idx 키에대해 값 업데이트

kobert_final_result = [row for simi, row in max_group.values()]

In [99]:
len(kobert_final_result), result[5501]

(5518, (5501, 3101, array([[0.74056315]], dtype=float32)))

In [100]:
kobert_final_matching_list = []
for row in kobert_final_result:

    # 답변
    answer = answer_df.loc[row[0], 'answer']
    answer_intro_conclusion = answer_df.loc[row[0], 'answer_intro_conclusion']
    # 질문
    question = question_df.loc[row[1], 'question']
    # 유사도 점수
    similarity_score = row[2]
    # 질문 의도
    intent = answer_df.loc[row[0], 'intention']
    # 질병관련 정보
    dc = answer_df.loc[row[0], 'disease_category']
    dn = answer_df.loc[row[0], 'disease_name']

    kobert_final_matching_list.append({'disease_category': dc, 'disease_name': dn,
                                       'intention': intent, 'question': question,
                                       # 'answer_intro_conclusion': answer_intro_conclusion,
                                       'answer': answer,
                                       'similarity_score': similarity_score})

In [101]:
kobert_final_matching_df = pd.DataFrame(kobert_final_matching_list)
kobert_final_matching_df[['question', 'answer', 'similarity_score']]

,question,answer,similarity_score
0,약물 복용을 통해 두통을 미리 예방하는 방법을 알려주세요.,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,[[0.71982515]]
1,갈색세포증의 증상을 개선하기 위해 건강에 좋은 운동이나 영양을 신경써야 할까요?,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활...",[[0.727221]]
2,두통 예방을 위해 충분한 휴식이 중요한가요?,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,[[0.7377082]]
3,두통을 예방하기 위해 어떤 예방 방법이 있는지 알려주세요.,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...,[[0.7103399]]
4,두통을 예방하기 위한 생활 습관에는 어떤 것이 있는지 알려주세요.,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,[[0.68233]]
...,...,...,...
5513,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...,[[0.79728603]]
5514,두통이 자주 발생하는 것은 일반적인 현상인가요? 집에서 할 수 있는 간단한 치료 방...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,[[0.7807157]]
5515,두통의 치료는 어떤 생활습관을 가지는 것이 중요한가요?,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,[[0.7130239]]
5516,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,[[0.7706493]]


In [104]:
kobert_final_matching_df['question'][0]

'약물 복용을 통해 두통을 미리 예방하는 방법을 알려주세요.'

In [105]:
kobert_final_matching_df['answer'][0].split('. ')

['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다',
 '일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다',
 '컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다',
 '의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다',
 '책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다',
 '모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다',
 '긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다',
 '또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다',
 '올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.']

In [107]:
kobert_final_matching_df.to_csv('kobert_matching.csv')

In [108]:
kobert_final_matching_df = pd.read_csv('kobert_matching.csv', index_col=0)
kobert_final_matching_df

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,약물 복용을 통해 두통을 미리 예방하는 방법을 알려주세요.,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,[[0.71982515]]
1,응급질환,두통,예방,갈색세포증의 증상을 개선하기 위해 건강에 좋은 운동이나 영양을 신경써야 할까요?,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활...",[[0.727221]]
2,응급질환,두통,예방,두통 예방을 위해 충분한 휴식이 중요한가요?,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,[[0.7377082]]
3,응급질환,두통,예방,두통을 예방하기 위해 어떤 예방 방법이 있는지 알려주세요.,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...,[[0.7103399]]
4,응급질환,두통,예방,두통을 예방하기 위한 생활 습관에는 어떤 것이 있는지 알려주세요.,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,[[0.68233]]
...,...,...,...,...,...,...
5513,응급질환,두통,치료,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...,[[0.79728603]]
5514,응급질환,두통,치료,두통이 자주 발생하는 것은 일반적인 현상인가요? 집에서 할 수 있는 간단한 치료 방...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,[[0.7807157]]
5515,응급질환,두통,치료,두통의 치료는 어떤 생활습관을 가지는 것이 중요한가요?,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,[[0.7130239]]
5516,응급질환,두통,치료,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,[[0.7706493]]


### (2)-1 KoSentence-BERT 모델로 문장 유사도 다시 구해보기

In [109]:
from sentence_transformers import SentenceTransformer, util

# KoSentence-BERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [110]:
# 임베딩 구하는 함수 만들기
def embedding_KR_SBERT(sentence):
    result = model.encode(sentence)
    
    return result

In [112]:
# 질문과 답변에 대한 임베딩 계산
question_embeddings_KR_SBERT = [embedding_KR_SBERT(q) for q in question_df['question']] 

In [113]:
answer_embeddings_KR_SBERT = [embedding_KR_SBERT(a) for a in answer_df['answer']]

In [114]:
ic_answer_embeddings_KR_SBERT = [embedding_KR_SBERT(a) for a in answer_df['answer_intro_conclusion']]

In [121]:
# 후보 쌍에 대해 코싸인 유사도 계산
KR_SBERT_final_similarities = []
for a_idx, q_idx in final_candidate_pairs:
    # print(a_idx, q_idx)
    c_simi = cosine_similarity([answer_embeddings_KR_SBERT[a_idx]], [question_embeddings_KR_SBERT[q_idx]])
    KR_SBERT_final_similarities.append((a_idx, q_idx, c_simi))

len(KR_SBERT_final_similarities), KR_SBERT_final_similarities[1900]

(110360, (95, 162, array([[0.38487184]], dtype=float32)))

In [122]:
# intro+conclusion 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
ic_KR_SBERT_final_similarities = []
for a_idx, q_idx in final_candidate_pairs:
    # print(a_idx, q_idx)
    c_simi = cosine_similarity([ic_answer_embeddings_kobert[a_idx]], [question_embeddings_kobert[q_idx]])
    ic_KR_SBERT_final_similarities.append((a_idx, q_idx, c_simi))

len(ic_KR_SBERT_final_similarities), ic_KR_SBERT_final_similarities[1900]

(110360, (95, 162, array([[0.6645186]], dtype=float32)))

In [123]:
KR_SBERT_combine_similarities = []
for ic_s, s in zip(ic_KR_SBERT_final_similarities, KR_SBERT_final_similarities):
    a, b, ic_c = ic_s
    _, _, c = s

    combine_score = 0.4*ic_c + 0.6*c

    KR_SBERT_combine_similarities.append((a, b, combine_score))

KR_SBERT_final_similarities = KR_SBERT_combine_similarities
KR_SBERT_final_similarities[1900]

(95, 162, array([[0.49673057]], dtype=float32))

In [126]:
from collections import defaultdict

# 키에 대한 기본값 설정
# 어떤 키를 넣든 없으면 해당 기본값을 줌
max_group = defaultdict(lambda: (-1, None)) # max_group에 처음 접근할 때 키가 없으면 (-1(유사도), None(원래값))이 기본값

for row in KR_SBERT_final_similarities:
    a_idx, q_idx, simi = row
    if simi > max_group[a_idx][0]: # simi > 현재 할당되어있는 유사도 이면
        max_group[a_idx] = (simi, row) # 동일한 a_idx 키에대해 값 업데이트

KR_SBERT_final_result = [row for simi, row in max_group.values()]

In [128]:
len(KR_SBERT_final_result)

5518

In [129]:
KR_SBERT_final_matching_list = []
for row in KR_SBERT_final_result:

    # 답변
    answer = answer_df.loc[row[0], 'answer']
    answer_intro_conclusion = answer_df.loc[row[0], 'answer_intro_conclusion']
    # 질문
    question = question_df.loc[row[1], 'question']
    # 유사도 점수
    similarity_score = row[2]
    # 질문 의도
    intent = answer_df.loc[row[0], 'intention']
    # 질병관련 정보
    dc = answer_df.loc[row[0], 'disease_category']
    dn = answer_df.loc[row[0], 'disease_name']

    KR_SBERT_final_matching_list.append({'disease_category': dc, 'disease_name': dn,
                                         'intention': intent, 'question': question,
                                         # 'answer_intro_conclusion':answer_intro_conclusion,
                                         'answer': answer,
                                         'similarity_score': similarity_score})

In [130]:
KR_SBERT_final_matching_df = pd.DataFrame(KR_SBERT_final_matching_list)
KR_SBERT_final_matching_df[['question', 'answer', 'similarity_score']]

,question,answer,similarity_score
0,두통 예방을 위해 어떤 스트레칭이 도움이 될까요?,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,[[0.62894523]]
1,두통 예방을 위해 어떤 자세와 운동 습관을 가지면 좋을까요?,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활...",[[0.67311627]]
2,두통 예방을 위해 충분한 휴식이 중요한가요?,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,[[0.73985565]]
3,두통 예방을 위한 효과적인 방법들을 알려주세요.,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...,[[0.72741425]]
4,두통을 예방하기 위한 생활 습관에는 어떤 것이 있는지 알려주세요.,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,[[0.69896096]]
...,...,...,...
5513,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...,[[0.7733924]]
5514,두통의 원인과 상태에 따라 어떤 치료 방법을 선택해야 할까요?,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,[[0.7367816]]
5515,두통의 치료는 어떤 생활습관을 가지는 것이 중요한가요?,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,[[0.71166277]]
5516,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,[[0.71532214]]


In [133]:
KR_SBERT_final_matching_df['question'][0]

'두통 예방을 위해 어떤 스트레칭이 도움이 될까요?'

In [134]:
KR_SBERT_final_matching_df['answer'][0].split('. ')

['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다',
 '일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다',
 '컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다',
 '의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다',
 '책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다',
 '모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다',
 '긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다',
 '또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다',
 '올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.']

In [135]:
KR_SBERT_final_matching_df.to_csv('KR_SBERT_matching.csv')

In [136]:
KR_SBERT_final_matching_df = pd.read_csv('KR_SBERT_matching.csv', index_col=0)
KR_SBERT_final_matching_df

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,두통 예방을 위해 어떤 스트레칭이 도움이 될까요?,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,[[0.62894523]]
1,응급질환,두통,예방,두통 예방을 위해 어떤 자세와 운동 습관을 가지면 좋을까요?,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활...",[[0.67311627]]
2,응급질환,두통,예방,두통 예방을 위해 충분한 휴식이 중요한가요?,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,[[0.73985565]]
3,응급질환,두통,예방,두통 예방을 위한 효과적인 방법들을 알려주세요.,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...,[[0.72741425]]
4,응급질환,두통,예방,두통을 예방하기 위한 생활 습관에는 어떤 것이 있는지 알려주세요.,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,[[0.69896096]]
...,...,...,...,...,...,...
5513,응급질환,두통,치료,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...,[[0.7733924]]
5514,응급질환,두통,치료,두통의 원인과 상태에 따라 어떤 치료 방법을 선택해야 할까요?,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,[[0.7367816]]
5515,응급질환,두통,치료,두통의 치료는 어떤 생활습관을 가지는 것이 중요한가요?,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,[[0.71166277]]
5516,응급질환,두통,치료,최근에 두통이 자주 발생하는 경우 어떤 종류의 두통인지 정확한 진단을 받을 수 있을까요?,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,[[0.71532214]]
